In [1]:
# Import necessary libraries
import os
import cv2
import json
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Constants
IMAGE_DIR = "images"
# IMAGE_SIZE = (100, 100)
# MobileNetV2 input size
IMAGE_SIZE = (224, 224)
MODEL_PATH = "model/face_cnn_model.h5"
LABEL_MAP_PATH = "model/label_map.json"

In [3]:
# Load face database into dictionary
def load_face_database(image_dir):
    database = {
        "Kobe": [],
        "Freeman": [],
        "Caleb": []
    }
    for file in os.listdir(image_dir):
        path = os.path.join(image_dir, file)
        if "kobe" in file.lower():
            database["Kobe"].append(path)
        elif "freeman" in file.lower():
            database["Freeman"].append(path)
        elif "caleb" in file.lower():
            database["Caleb"].append(path)
    return database

In [4]:
# Extract face from image
def extract_face(img):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        return cv2.resize(face, IMAGE_SIZE)
    return None

In [5]:
# Preprocess images
def load_data(image_dir):
    database = load_face_database(image_dir)
    X, y = [], []
    label_map = {name: idx for idx, name in enumerate(database)}
    for label, paths in database.items():
        for path in paths:
            img = cv2.imread(path)
            face = extract_face(img)
            if face is not None:
                X.append(face / 255.0)
                y.append(label_map[label])
    return np.array(X), to_categorical(y), label_map

In [6]:
# Load data
X, y, label_map = load_data(IMAGE_DIR)

In [7]:
# Save label_map to JSON
os.makedirs("model", exist_ok=True)
with open(LABEL_MAP_PATH, "w") as file:
    json.dump(label_map, file)

In [8]:
# Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     brightness_range=[0.6, 1.4],
#     horizontal_flip=True,
#     fill_mode='nearest'
# )
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True
)

In [9]:
# Transfer Learning Model: MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(label_map), activation='softmax')(x)

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
# Freeze base model
for layer in base_model.layers[-20:]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(datagen.flow(X, y, batch_size=8), epochs=20)
model.save(MODEL_PATH)
print("Model trained and saved.")

Epoch 1/20
1/2 ━━━━━━━━━━━━━━━━━━━━ 13s 14s/step - accuracy: 1.0000 - loss: 0.5464

C:\Users\Usuario\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 261ms/step - accuracy: 0.6296 - loss: 0.9929
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - accuracy: 0.7778 - loss: 0.5695
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - accuracy: 0.6296 - loss: 0.9562
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 0.6296 - loss: 0.8885
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.3704 - loss: 1.9073  
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.2222 - loss: 3.5253  
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.6296 - loss: 0.7441
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.4213 - loss: 1.4725 
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 0.3704 - loss: 1.9911  
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.4444 - loss: 0.8813  
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5370 - loss: 1.0450
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4213 - loss: 

Model trained and saved.


In [13]:
# Load trained model and label map
model = load_model(MODEL_PATH)
with open(LABEL_MAP_PATH, "r") as f:
    label_map = json.load(f)
# invert to {0: 'Name'}
label_map = {v: k for k, v in label_map.items()}  

In [20]:
# Real-time recognition
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)

# threshold = 0.7
print("Press 'q' to quit...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    faces = face_cascade.detectMultiScale(frame, 1.3, 5)

    for (x, y, w, h) in faces:
        roi = frame[y:y+h, x:x+w]
        face = cv2.resize(roi, IMAGE_SIZE)
        face = np.expand_dims(face / 255.0, axis=0)

        prediction = model.predict(face, verbose=0)[0]
        label_index = np.argmax(prediction)
        confidence = prediction[label_index]
        name = label_map[label_index] #if confidence >= threshold else "Unknown"

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"{name} ({confidence:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.imshow("Face Recognition - MobileNetV2", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Press 'q' to quit...


In [15]:
# # # Define CNN model
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(100,100,3)),
#     MaxPooling2D(2,2),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Conv2D(128, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dense(128, activation='relu'),
#     # Reduce overfitting
#     Dropout(0.5),  
#     Dense(len(label_map), activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# # Train model
# model.fit(datagen.flow(X, y, batch_size=8), epochs=50)
# model.save(MODEL_PATH)
# print("Model trained and saved.")

In [17]:
# # Reload model and label_map for prediction
# with open(LABEL_MAP_PATH, "r") as file:
#     label_map = json.load(file)

# # Convert to {0: "Kobe", 1: "Freeman", 2: "Caleb"}
# label_map = {v: k for k, v in label_map.items()}

In [18]:
# # Load OpenCV face detector
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# cap = cv2.VideoCapture(0)
# print("Press 'q' to Quit.")

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     faces = face_cascade.detectMultiScale(frame, 1.3, 5)

#     for (x, y, w, h) in faces:
#         roi = frame[y:y+h, x:x+w]
#         img = cv2.resize(roi, IMAGE_SIZE)
#         img = np.expand_dims(img / 255.0, axis=0)

#         prediction = model.predict(img, verbose=0)[0]
#         label_index = np.argmax(prediction)
#         confidence = prediction[label_index]
#         name = label_map[label_index]

#         threshold = 0.7
#         if confidence < threshold:
#             name = "Unknown"

#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
#         # text = f"{name} ({confidence:.2f})"
#         # cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
#         cv2.putText(frame, f"{name} ({confidence:.2f})", (x, y - 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
#     cv2.imshow("Face Recognition", frame)
#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break

# cap.release()
# cv2.destroyAllWindows()